In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

코랩 사용하시는 분들은 아래의 코드를 실행해주세요!

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import os
# os.getcwd()

코랩 사용시 본인 구글 드라이브 경로는  
/content/drive/MyDrive/...  
입니다.

In [4]:
import os

In [5]:
path = '/content/drive/MyDrive/DSL/2022-2/220804 supervised_learning/data'    # 과제 데이터셋이 저장된 경로를 더 작성해주세요
os.chdir(path)

In [6]:
os.getcwd()

'/content/drive/MyDrive/DSL/2022-2/220804 supervised_learning/data'

사용할 데이터 셋 :  전력 사용량 예측 모델

아래의 과정에서 sklearn의 Pipeline의 사용은 필수는 아닙니다. 선호와 필요에 따라 사용하실 분들은 사용하셔도 좋습니다.

# 데이터

건물 정보와 기후 정보를 활용한 전력사용량 예측 데이터 중에서 

환경 변수(기온, 풍속, 습도, 강수량, 일조)에 대해 전력 사용량을 선형회귀 모델을 사용해서 예측해보고자 합니다.

In [7]:
var_name = ['Temp', 'Wind', 'Moisture', 'Precipitation', 'Sun']
target = 'Wh'

In [8]:
def parsing(data: pd.DataFrame):
    x = data.iloc[:, 3:8]
    x.columns = var_name
    y = data.iloc[:, 2]
    y.name = target
    
    return x, y

In [11]:
def load_data():
    data = pd.read_csv('/content/drive/MyDrive/DSL/2022-2/220804 supervised_learning/data/train.csv', encoding='cp949')
    data = data[: 100]
    x, y = parsing(data)    
    cut = int(round(len(x)*0.88))
    tr_x = x[:cut]
    tr_y = y[:cut]
    te_x = x[cut:]
    te_y = y[cut:]
    return tr_x, tr_y, te_x, te_y

In [12]:
train_x, train_y, test_x, test_y = load_data()

## 데이터 확인 

주의! 최종 예측하는 단계 이전에는 현재 가지고 있는 데이터, 즉 train data만을 이용하여 작업을 진행해야 합니다.

현재 가지고 있는 데이터 train_x를 보고 전력 사용량을 예측하는데에 사용할 변수를 선택해주세요.

기초세션에서 다뤘던 내용을 참고하셔서 최적의 변수 조합을 찾아보도록 합시다.

이 때 선택해야하는 변수는 하나일 수도 있고, 여러개의 변수를 사용하여도 좋습니다.

In [15]:
# 데이터 셋을 둘러보는 코드 작성하시고, 왜 좋은지 한번 둘러보세요!
train_x

,Temp,Wind,Moisture,Precipitation,Sun
0,17.6,2.5,92.0,0.8,0.0
1,17.7,2.9,91.0,0.3,0.0
2,17.5,3.2,91.0,0.0,0.0
3,17.1,3.2,91.0,0.0,0.0
4,17.0,3.3,92.0,0.0,0.0
...,...,...,...,...,...
83,23.9,2.8,68.0,0.0,0.6
84,25.0,3.1,63.0,0.0,0.8
85,26.0,3.1,60.0,0.0,1.0
86,26.4,3.8,60.0,0.0,1.0


In [14]:
train_y

0     8179.056
1     8135.640
2     8107.128
3     8048.808
4     8043.624
        ...   
83    8386.416
84    8435.664
85    8591.184
86    8638.488
87    8663.112
Name: Wh, Length: 88, dtype: float64

In [24]:
train_x= train_x[['Temp','Moisture']]

In [25]:
train_x

,Temp,Moisture
0,17.6,92.0
1,17.7,91.0
2,17.5,91.0
3,17.1,91.0
4,17.0,92.0
...,...,...
83,23.9,68.0
84,25.0,63.0
85,26.0,60.0
86,26.4,60.0


정답은 없습니다! 

다만 최종 선택한 변수(들)이 왜 선택받았는지 간단히 알려주세요.<br>
답변 : 날씨가 중 에어컨 소비와, 난방에 의해 변화되는 전력량이 클 것이다라고 판단하여 기온과 습도 변수를 선택하였습니다. 

## 전처리

위에서 선택한 변수들(feature)를 가지고 한번 전력 사용량을 예측하는 모델을 만들어 보기에 앞서,

**train 데이터의 feature의 전처리**를 진행해보려고 합니다. StandardScaler를 사용하셔도 좋고, MinMaxScaler를 사용하셔도 좋습니다! 

또한 PolynomialFeatures를 이용하거나 직접 feature를 계산하여 고차원의 feature를 만드셔도 좋습니다. 이 또한 정답은 없습니다!

이 때 최종 train_x의 shape은 (n_samples, n_features)가 되도록 해주세요.<br>
ex. 4개의 변수를 가진 1000개의 샘플의 경우 train_x.shape은 (1000,4)가 되면 됩니다.

특히 하나의 변수만을 사용하시는 분들은 1D가 아닌 2D가 되도록 신경써주세요<br>
ex. train_x.shape이 (1000,)이 아닌 (1000,1)입니다!

In [27]:
# 사용할 패키지를 import!
from sklearn.preprocessing import MinMaxScaler

In [35]:
num_sample = len(train_x)
num_axis = len(train_x.columns)
print(num_sample, num_axis)

88 2


In [36]:
train_x = train_x.to_numpy().reshape(num_sample,num_axis)

In [ ]:
train_x

In [60]:
scaler = MinMaxScaler()
scaler = scaler.fit(train_x)
processed_train_x = scaler.transform(train_x)
processed_train_x

array([[0.20895522, 0.98148148],
       [0.21641791, 0.96296296],
       [0.20149254, 0.96296296],
       [0.17164179, 0.96296296],
       [0.1641791 , 0.98148148],
       [0.15671642, 1.        ],
       [0.14179104, 0.94444444],
       [0.15671642, 0.87037037],
       [0.2238806 , 0.75925926],
       [0.3358209 , 0.62962963],
       [0.42537313, 0.42592593],
       [0.54477612, 0.24074074],
       [0.61940299, 0.18518519],
       [0.61940299, 0.05555556],
       [0.65671642, 0.        ],
       [0.67164179, 0.01851852],
       [0.71641791, 0.        ],
       [0.65671642, 0.03703704],
       [0.58955224, 0.05555556],
       [0.48507463, 0.09259259],
       [0.37313433, 0.22222222],
       [0.28358209, 0.2962963 ],
       [0.23134328, 0.42592593],
       [0.17910448, 0.53703704],
       [0.13432836, 0.64814815],
       [0.10447761, 0.7037037 ],
       [0.08208955, 0.72222222],
       [0.06716418, 0.72222222],
       [0.05223881, 0.74074074],
       [0.        , 0.74074074],
       [0.

# 모델링

## 학습 

이제 모델링을 해봅시다.

앞서 다룬 여러 regression 모델들(Linear regression, Ridge Regression, Lasso Regression, ..)을 이용해서 fit을 진행해보도록 합시다.

In [40]:
# 사용할 패키지를 import!
from sklearn.linear_model import LinearRegression

In [51]:
# 선형회귀 모델 생성 후 fitting 시킵니다.
linear_model = LinearRegression().fit(processed_train_x, train_y)

linear_beta1 = linear_model.coef_[0]
linear_beta2 = linear_model.coef_[1]
linear_intercept  = linear_model.intercept_

print(f"Beta :{linear_beta1: .4f} {linear_beta2: .4f}")
print(f"Intercept : {linear_intercept: .4f}")

Beta : 1021.5279  466.8392
Intercept :  7450.3920


## 추정

이제 train_x를 이용하여 전력사용량을 추정해보도록 합시다.

In [52]:
esti_y =  linear_model.predict(processed_train_x)


# Loss 및 시각화

앞서 fitting하여 얻은 모델을 이용하여 추정한 값으로 MSE를 측정해봅시다. 변수를 하나만 사용하신 분들은 xy 평면에 한번 같이 시각화도 해보셔도 좋습니다.

In [53]:
def mse(pred, true):
    return np.mean((pred-true)**2)

In [54]:
train_mse = mse(esti_y, train_y)
print(f"Linear {train_mse:.5f}")

Linear 3150.62873


In [77]:
train_x[:,0]

array([17.6, 17.7, 17.5, 17.1, 17. , 16.9, 16.7, 16.9, 17.8, 19.3, 20.5,
       22.1, 23.1, 23.1, 23.6, 23.8, 24.4, 23.6, 22.7, 21.3, 19.8, 18.6,
       17.9, 17.2, 16.6, 16.2, 15.9, 15.7, 15.5, 14.8, 15. , 15.5, 16. ,
       16.9, 17.3, 17.5, 16.3, 17.5, 20.4, 20.9, 21.7, 21.2, 20.6, 20.3,
       19.9, 19.6, 19.3, 19. , 18.4, 18.3, 18.2, 18.2, 18.1, 18.1, 17.6,
       18.3, 19.7, 20.8, 22. , 23.1, 24.4, 26.1, 26.8, 27.9, 28.2, 28.1,
       26.1, 23.8, 21. , 20.5, 20.4, 20.1, 20.1, 20. , 20.1, 20.4, 20.5,
       20.3, 20.1, 20.1, 20.1, 20.6, 22.4, 23.9, 25. , 26. , 26.4, 25.4])

In [86]:
from mpl_toolkits.mplot3d import Axes3D

In [92]:
train_x.shape

(88, 2)

In [93]:
def visualize(model, x, y):
    beta_1 = model.coef_[0]
    beta_2 = model.coef_[1]
    intercept = model.intercept_
    
    x1_axis = np.linspace(np.min(x[:,0]), np.max(x[:,0]), 100)
    x2_axis = np.linspace(np.min(x[:,1]), np.max(x[:,1]), 100)
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot( projection='3d')
    ax.scatter(x[:,0],x[:,1], y, c=color, marker='o', s=15, cmap='Greens')
    plt.legend()
    plt.show()

In [94]:
import matplotlib.pyplot as plt

# 예측

테스트셋에 대해서 한번 **예측**해보도록 합시다.

이미 fitting된 모델 (그리고 scaler를 사용하신다면 scaler)에 대해 **fit을 하지 말고** 단순히 predict만 진행해보도록 합시다. 

scaler는 transform만 하시고 model은 predict을 진행해주세요

In [56]:
# transform을 하시는 분들은 실행해주세요

test_x =  test_x[['Temp','Moisture']]

In [58]:

num_sample = len(test_x)
num_axis = len(test_x.columns)
test_x= test_x.to_numpy().reshape(num_sample,num_axis) 


In [61]:
processed_test_x = scaler.transform(test_x)
processed_test_x

array([[0.70895522, 0.57407407],
       [0.62686567, 0.66666667],
       [0.59701493, 0.7037037 ],
       [0.51492537, 0.75925926],
       [0.49253731, 0.81481481],
       [0.44776119, 0.87037037],
       [0.41044776, 0.92592593],
       [0.38059701, 0.96296296],
       [0.36567164, 0.94444444],
       [0.35074627, 0.94444444],
       [0.3358209 , 0.94444444],
       [0.3358209 , 0.94444444]])

In [62]:
# y1_hat에 predict값 할당

pred_y = linear_model.predict(processed_test_x)

# 성능 평가

한번 학습한 모델에 대해 test_y를 이용하여 mse loss를 측정해도도록 합시다.

In [63]:
def score(train_mse, test_mse):
    print(f"{test_mse/train_mse: .4f}")

In [64]:
test_mse = mse(pred_y, test_y)
print(f"Linear {test_mse:.5f}")

Linear 95713.46656


In [65]:
score(train_mse,test_mse)

 30.3792


In [68]:
visualize(linear_model, test_x, test_y)

# 그 이후

이제 여러분들은 한번 모델을 만들어보고 fitting한 다음 예측까지 해보셨습니다.

이제 개선하기 위해 1.데이터부터 5.성능평가 중 수정, 변경하실 부분을 업데이트 해서  최적의 모델을 찾아보세요!

현재 사용하고 있는 scaler, model부터 전처리과정 기타 여러 조건을 조작해보시고 어떤 효과가 있는지 한번 보시고 알려주세요:D

# scalaer를 min max 스케일러에서, standard scaler로 바꾸고 마찬가지로 다항 회귀 모델을 통해 학습시켜보고 성능을 평가해보겠습니다!

In [96]:
train_x, train_y, test_x, test_y = load_data()

In [97]:
train_x= train_x[['Temp','Moisture']]

In [98]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler = scaler.fit(train_x)
processed_data = scaler.transform(train_x)
processed_data

array([[-8.09422591e-01,  1.32132616e+00],
       [-7.78799552e-01,  1.25833770e+00],
       [-8.40045630e-01,  1.25833770e+00],
       [-9.62537785e-01,  1.25833770e+00],
       [-9.93160823e-01,  1.32132616e+00],
       [-1.02378386e+00,  1.38431463e+00],
       [-1.08502994e+00,  1.19534924e+00],
       [-1.02378386e+00,  9.43395387e-01],
       [-7.48176514e-01,  5.65464610e-01],
       [-2.88830933e-01,  1.24545370e-01],
       [ 7.86455312e-02, -5.68327722e-01],
       [ 5.68614150e-01, -1.19821235e+00],
       [ 8.74844537e-01, -1.38717774e+00],
       [ 8.74844537e-01, -1.82809698e+00],
       [ 1.02795973e+00, -2.01706237e+00],
       [ 1.08920581e+00, -1.95407390e+00],
       [ 1.27294404e+00, -2.01706237e+00],
       [ 1.02795973e+00, -1.89108544e+00],
       [ 7.52352383e-01, -1.82809698e+00],
       [ 3.23629841e-01, -1.70212005e+00],
       [-1.35715740e-01, -1.26120081e+00],
       [-5.03192204e-01, -1.00924696e+00],
       [-7.17553475e-01, -5.68327722e-01],
       [-9.

In [99]:
# 선형회귀 모델 생성 후 fitting 시킵니다.
linear_model = LinearRegression().fit(processed_train_x, train_y)

linear_beta1 = linear_model.coef_[0]
linear_beta2 = linear_model.coef_[1]
linear_intercept  = linear_model.intercept_

print(f"Beta :{linear_beta1: .4f} {linear_beta2: .4f}")
print(f"Intercept : {linear_intercept: .4f}")

Beta : 1021.5279  466.8392
Intercept :  7450.3920


In [100]:
esti_y =  linear_model.predict(processed_train_x)


In [101]:
train_mse = mse(esti_y, train_y)
print(f"Linear {train_mse:.5f}")

Linear 3150.62873


In [102]:

test_x =  test_x[['Temp','Moisture']]
num_sample = len(test_x)
num_axis = len(test_x.columns)
test_x= test_x.to_numpy().reshape(num_sample,num_axis) 


In [103]:
processed_test_x = scaler.transform(test_x)
processed_test_x

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


array([[ 1.242321  , -0.06442002],
       [ 0.90546758,  0.2505223 ],
       [ 0.78297542,  0.37649922],
       [ 0.446122  ,  0.56546461],
       [ 0.35425288,  0.75443   ],
       [ 0.17051465,  0.94339539],
       [ 0.01739945,  1.13236078],
       [-0.1050927 ,  1.2583377 ],
       [-0.16633878,  1.19534924],
       [-0.22758486,  1.19534924],
       [-0.28883093,  1.19534924],
       [-0.28883093,  1.19534924]])

In [104]:
pred_y = linear_model.predict(processed_test_x)

In [105]:
test_mse = mse(pred_y, test_y)
print(f"Linear {test_mse:.5f}")


Linear 368228.60931


In [106]:
score(train_mse,test_mse)

 116.8746


#min max scaler 보다 standard scaler로 학습시켰을 때 score가 4배 정도로 훨씬 더 좋게 나왔습니다! 지금 독립변수로 삼은 피처 온도와 습도의 경우 min max간의 차이가 얼마 차이가 나지 않아 학습하는데 안좋은 영향을 끼쳤던 것 같습니다! 

고생하셨습니다!